In [77]:
import pandas as pd
import numpy as np

In [78]:
data = pd.read_csv('./weather_classification_data.csv')
data.head()

,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Weather Type
0,14.0,73,9.5,82.0,partly cloudy,1010.82,2,Winter,3.5,inland,Rainy
1,39.0,96,8.5,71.0,partly cloudy,1011.43,7,Spring,10.0,inland,Cloudy
2,30.0,64,7.0,16.0,clear,1018.72,5,Spring,5.5,mountain,Sunny
3,38.0,83,1.5,82.0,clear,1026.25,7,Spring,1.0,coastal,Sunny
4,27.0,74,17.0,66.0,overcast,990.67,1,Winter,2.5,mountain,Rainy


In [79]:
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

oe = OrdinalEncoder()
data['Cloud Cover'] = oe.fit_transform(data['Cloud Cover'].values.reshape((-1,1)))
le = LabelEncoder()
data['Season'] = le.fit_transform(data['Season'].values.reshape((-1,1)))
data['Location'] = le.fit_transform(data['Location'].values.reshape((-1,1)))
data['Weather Type'] = le.fit_transform(data['Weather Type'].values.reshape((-1,1)))
data.head()

/Users/bhagya/Library/Python/3.9/lib/python/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/bhagya/Library/Python/3.9/lib/python/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/bhagya/Library/Python/3.9/lib/python/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Weather Type
0,14.0,73,9.5,82.0,3.0,1010.82,2,3,3.5,1,1
1,39.0,96,8.5,71.0,3.0,1011.43,7,1,10.0,1,0
2,30.0,64,7.0,16.0,0.0,1018.72,5,1,5.5,2,3
3,38.0,83,1.5,82.0,0.0,1026.25,7,1,1.0,0,3
4,27.0,74,17.0,66.0,2.0,990.67,1,3,2.5,2,1


In [80]:
y = data['Weather Type']
X = data.drop(['Weather Type'], axis = 1)
X.shape, y.shape

((13200, 10), (13200,))

In [81]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler

std = StandardScaler()
X_scaled = std.fit_transform(X)
y_encoded = to_categorical(y)
X_scaled.shape, y_encoded.shape

((13200, 10), (13200, 4))

In [82]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

In [83]:
import keras
from keras.layers import Dense, Dropout
from keras.models import Sequential

In [84]:
model = Sequential()
model.add(Dense(32, input_shape = (X_train.shape[1],), activation = 'sigmoid'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128, activation='sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(512, activation='softmax'))
model.add(Dropout(0.4))
model.add(Dense(128, activation='softmax'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(y_train.shape[1], activation='sigmoid'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

/Users/bhagya/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_35 (Dense)                │ (None, 32)             │           352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_25 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_26 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_38 (Dense)                │ (None, 512)            │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_27 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_39 (Dense)                │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_28 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_40 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_29 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_41 (Dense)                │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 151,012 (589.89 KB)

 Trainable params: 151,012 (589.89 KB)

 Non-trainable params: 0 (0.00 B)

In [85]:
model.fit(X_train,
          y_train,
          epochs =200, 
          verbose = 1,
          batch_size = 32,
          validation_split = 0.2)

Epoch 1/200
264/264 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.2384 - loss: 1.3868 - val_accuracy: 0.2472 - val_loss: 1.3868
Epoch 2/200
264/264 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2678 - loss: 1.3850 - val_accuracy: 0.4228 - val_loss: 1.3144
Epoch 3/200
264/264 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3712 - loss: 1.2589 - val_accuracy: 0.6075 - val_loss: 1.0898
Epoch 4/200
264/264 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4295 - loss: 1.1513 - val_accuracy: 0.5985 - val_loss: 1.0225
Epoch 5/200
264/264 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4950 - loss: 1.0698 - val_accuracy: 0.6473 - val_loss: 0.9457
Epoch 6/200
264/264 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5815 - loss: 0.9918 - val_accuracy: 0.6918 - val_loss: 0.8117
Epoch 7/200
264/264 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6410 - loss: 0.8794 - val_accuracy: 0.7382 - val_loss: 0.7362
Epoch 8/200
264/264 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6868 - loss: 0.8107 - val_accu

In [86]:
model.evaluate(X_test, y_test)

83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 552us/step - accuracy: 0.9000 - loss: 0.2128


[0.210933655500412, 0.904924213886261]